In [12]:
import random
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

# Step 1: Load Australia's boundary as a GeoDataFrame
# You can download a shapefile or GeoJSON of Australia's boundaries
# Here, we're assuming you have it stored as 'australia_boundary.geojson'
australia = gpd.read_file('australia_boundary.geojson')

# Step 2: Generate random points within Australia's bounding box
# Bounding box for Australia: latitude (-44, -10), longitude (113, 154)
def generate_random_clinic_within_australia():
    while True:
        latitude = random.uniform(-44, -10)
        longitude = random.uniform(113, 154)
        clinic = Point(longitude, latitude)
        # Check if the clinic falls within Australia's boundary
        if australia.contains(clinic).any():
            return clinic

# Step 3: Generate 50 random clinics
clinics = [generate_random_clinic_within_australia() for _ in range(50)]

# Convert to GeoDataFrame for easier handling
clinics_gdf = gpd.GeoDataFrame(geometry=clinics, crs=australia.crs)

supplier = [generate_random_clinic_within_australia() for _ in range(20)]

# Convert to GeoDataFrame for easier handling
supplier_gdf = gpd.GeoDataFrame(geometry=supplier, crs=australia.crs)


In [13]:
# load pickle file into clinic_name
import pickle
with open('clinic_value_set.pkl', 'rb') as f:
    clinic_name = pickle.load(f)

In [14]:
clinic_name_list = []

for i in clinic_name:
    clinic_name_list.append(i)
    
clinic_name_list

['N21701',
 'Q4858',
 'N2511',
 'N21322',
 'NZ9811',
 'N20255',
 'N21211',
 'Q4647',
 'N20501',
 'N20652',
 'N21221',
 'N21533',
 'N28555',
 'NZ0611',
 'NZ0618',
 'N2654',
 'N0001',
 'N20022',
 'N21131',
 'N22323',
 'N26166',
 'N2059',
 'N20000',
 'N2892',
 'Q4410',
 'N20221',
 'N2433',
 'Q4601\n',
 'Q2222',
 'Q40400',
 'Q4161',
 'Q43353',
 'N20224',
 'Q4587',
 'N21341',
 'NZ2120',
 'NZ2313',
 'NZ21044',
 'N22911',
 'V302994',
 'V31166',
 'V31616',
 'N20761',
 'V3340',
 'NZ10111',
 'NZ4140',
 'N22002',
 'NZ1071',
 'Q4521',
 'N2667']

In [15]:
import pandas as pd
from shapely.geometry import Point
from geopy.distance import geodesic

# Step 1: Generate unique names for clinics and suppliers
clinics_gdf['name'] = clinic_name_list
supplier_gdf['name'] = [f'Supplier {i+1}' for i in range(len(supplier_gdf))]

# Step 2: Function to calculate geodesic distance
def calculate_distance(clinic_point, supplier_point):
    # Extract latitude and longitude for both points
    clinic_coords = (clinic_point.y, clinic_point.x)
    supplier_coords = (supplier_point.y, supplier_point.x)
    # Calculate distance in kilometers
    return geodesic(clinic_coords, supplier_coords).kilometers

# Step 3: Create distance matrix
distance_matrix = []

for clinic in clinics_gdf.geometry:
    distances = [
        calculate_distance(clinic, supplier) for supplier in supplier_gdf.geometry
    ]
    distance_matrix.append(np.round(distances,2))

# Step 4: Convert to DataFrame and label with clinic and supplier names
distance_df = pd.DataFrame(
    distance_matrix, 
    columns=supplier_gdf['name'], 
    index=clinics_gdf['name']
)

# Step 5: Display or save the result
print(distance_df)
distance_df.to_csv('clinic_supplier_distance_matrix.csv')


name     Supplier 1  Supplier 2  Supplier 3  Supplier 4  Supplier 5  \
name                                                                  
N21701        81.61     1410.88     1085.89     1661.56     2096.33   
Q4858        587.04     1163.31      572.20     1809.09     2374.61   
N2511       1580.95     3003.21     2498.53     1115.63      837.42   
N21322      1118.69     2329.90     1448.68      640.59     1274.85   
NZ9811      1730.15     1509.26      576.69     2360.67     3070.87   
N20255      2153.33     3574.84     2927.43     1094.63      422.41   
N21211       626.39     1467.89     1661.21     2173.75     2429.87   
Q4647       2115.26     3286.90     2267.57      504.70      988.01   
N20501       709.81     1736.86      895.35     1239.96     1842.81   
N20652      1112.25      338.28      910.22     2629.86     3158.09   
N21221       557.17     1906.78     1697.35     1595.83     1807.57   
N21533       100.40     1348.65     1168.27     1793.46     2194.75   
N28555

In [16]:
import folium

# Step 1: Create a base map centered on Australia
# Adjust the location and zoom_start to suit your needs
map_center = [-25.2744, 133.7751]  # Approximate center of Australia
m = folium.Map(location=map_center, zoom_start=5)

# Step 2: Add clinic points to the map
for idx, row in clinics_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"Clinic: {row['name']}"
    ).add_to(m)

# Step 3: Add supplier points to the map
for idx, row in supplier_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        popup=f"Supplier: {row['name']}"
    ).add_to(m)

# Step 4: Save the map as an HTML file or display it
m.save('clinic_supplier_map.html')

# To display in a Jupyter notebook
m


In [17]:
clinics_gdf['latitude'] = clinics_gdf.geometry.y
clinics_gdf['longitude'] = clinics_gdf.geometry.x
clinics_gdf.drop(columns='geometry').to_csv('clinics_australia.csv', index=False)


In [ ]:
clinics_gdf.plot(marker='o', color='blue', figsize=(10, 10))


In [ ]:
clinics_gdf['name'] = [f'Clinic_{i+1}' for i in range(len(clinics_gdf))]
